In [1]:
import pandas as pd
import numpy as np
# from warnings import warn

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]


data = pd.read_csv('./data/titanic.csv', usecols=use_cols)
data.head(3)
print(data.shape)

(891, 6)


In [13]:
pd.Series(data.Fare.unique()).sort_values()
#data.Fare.unique()

104      0.0000
163      4.0125
245      5.0000
152      6.2375
240      6.4375
         ...   
164    227.5250
75     247.5208
148    262.3750
23     263.0000
127    512.3292
Length: 248, dtype: float64

### Detect by arbitrary boundary
#### identify outliers based on arbitrary boundaries

In [39]:
def outlier_detect_arbitrary(data,col,upper_fence,lower_fence):
    '''
    identify outliers based on arbitrary boundaries passed to the function.
    '''
    para = (upper_fence,lower_fence)
    tmp = pd.concat([data[col] > upper_fence, data[col] < lower_fence],axis = 1)
    outlier_index = tmp.any(axis = 1)
    print('Num of outliers detected: ',outlier_index.value_counts()[1])
    print('Proportion of outlier detected :',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index, para

In [40]:
index,para = outlier_detect_arbitrary(data=data,col='Fare',upper_fence=300,lower_fence=5)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outliers detected:  19
Proportion of outlier detected : 0.02132435465768799
Upper bound: 300 
Lower bound: 5


In [42]:
# check the 19 found outliers
data.loc[index, 'Fare'].sort_values()

179      0.0000
806      0.0000
732      0.0000
674      0.0000
633      0.0000
597      0.0000
815      0.0000
466      0.0000
481      0.0000
302      0.0000
277      0.0000
271      0.0000
263      0.0000
413      0.0000
822      0.0000
378      4.0125
679    512.3292
737    512.3292
258    512.3292
Name: Fare, dtype: float64